## Init and import

In [1]:
# @title Imports

import collections
import concurrent.futures
import datetime

# from google.colab import widgets
from IPython import display

from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import measurements as measurements_lib
from concordia.language_model import gpt_model
from concordia.language_model import gcloud_model
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.ERROR, filename='components_testing.log')
logger = logging.getLogger('ollama')

In [2]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

In [18]:
from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    # model_name='llama2:70b',
    model_name='mixtral'
    # streaming=True
)

# import dotenv
# import os
# dotenv.load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')

# from concordia.language_model import gpt_model
# model = gpt_model.GptLanguageModel(
#     api_key=api_key,
#     model_name='gpt-4',
# )

In [19]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])


In [20]:
measurements = measurements_lib.Measurements()

In [21]:
import importlib
import custom_components.quiz_metric as qm
importlib.reload(qm)

context = """Alice is a carpenter. She has been a carpenter for her whole life. She has never learned to code. 
She doesn't know anything about data structures. She is a carpenter.
Alice is taking a quiz on data structures, just for fun, to test her knowledge. 
If Alice doesn't know the answer, she will guess and make a single choice.
How would Alice answer the following question?
"""

quiz_metric = qm.QuizMetric(model, "Alice", clock, "./custom_components/advanced_ds_questions.json", measurements=measurements, verbose=True)
quiz_metric.observe(context)

Question: Which of the following is not a property of a Red-Black Tree?, Agent answered incorrectly: If a node is red, then both its children are black
Question: In a B-Tree of order m, what is the maximum number of children a node can have?, Agent answered incorrectly: 2m-1
Question: Which of the following is a disadvantage of using hash tables?, Agent answered correctly: Handling of collisions
Question: In the context of a graph, what does 'DAG' stand for?, Agent answered correctly: Directed Acyclic Graph
Question: What is the worst-case time complexity of inserting an element in a Fibonacci Heap?, Agent answered incorrectly: O(n)
Question: Which algorithm is not typically used for graph traversal?, Agent answered correctly: Quick Sort
Question: What is the space complexity of a simple adjacency list representation of a graph with n vertices and e edges?, Agent answered incorrectly: O(n)
Question: In AVL trees, what is the maximum difference allowed between the heights of the left an

In [23]:
context = """Bob has a PhD in computer science. He has been interested in data structures for his whole life.
He knows everything about data structures. He is a computer scientist.
Bob is taking a quiz on data structures, just for fun, to test his knowledge.
If Bob doesn't know the answer, he will guess and make a single choice.
How would Bob answer the following question?"""

quiz_metric = qm.QuizMetric(model, "Bob", clock, "./custom_components/advanced_ds_questions.json", measurements=measurements, verbose=True)
quiz_metric.observe(context)

Question: Which of the following is not a property of a Red-Black Tree?, Agent answered incorrectly: The root is always black
Question: In a B-Tree of order m, what is the maximum number of children a node can have?, Agent answered incorrectly: 2m-1
Question: Which of the following is a disadvantage of using hash tables?, Agent answered correctly: Handling of collisions
Question: In the context of a graph, what does 'DAG' stand for?, Agent answered correctly: Directed Acyclic Graph
Question: What is the worst-case time complexity of inserting an element in a Fibonacci Heap?, Agent answered correctly: Amortized O(1)
Question: Which algorithm is not typically used for graph traversal?, Agent answered correctly: Quick Sort
Question: What is the space complexity of a simple adjacency list representation of a graph with n vertices and e edges?, Agent answered correctly: O(n + e)
Question: In AVL trees, what is the maximum difference allowed between the heights of the left and right subtrees